# Gradient Descent

### Predicting if a person would buy an insurance based on his age and affordability

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

2024-04-28 23:45:50.085468: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
df = pd.read_csv('insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


Split the data using train_test_split

In [5]:
from sklearn.model_selection import train_test_split

X = df[['age', 'affordibility']]
y = df['bought_insurance']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)

**Preprocessing:** Scale the data so that both age and affordibility are in same scaling range

In [6]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [7]:
X_test_scaled

,age,affordibility
2,0.47,1
10,0.18,1
21,0.26,0
11,0.28,1
14,0.49,1
9,0.61,1


**Model Building:** First build a model in keras/tensorflow and see what weights and bias values it comes up with. We will than try to reproduce same weights and bias in our plain python implementation of gradient descent.

In [8]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(X_train_scaled, y_train, epochs=1000)

Epoch 1/1000
1/1 [==============================] - 1s 1s/step - loss: 0.7113 - accuracy: 0.5000
Epoch 2/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.7110 - accuracy: 0.5000
Epoch 3/1000
1/1 [==============================] - 0s 43ms/step - loss: 0.7106 - accuracy: 0.5000
Epoch 4/1000
1/1 [==============================] - 0s 57ms/step - loss: 0.7102 - accuracy: 0.5000
Epoch 5/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.7098 - accuracy: 0.5000
Epoch 6/1000
1/1 [==============================] - 0s 19ms/step - loss: 0.7094 - accuracy: 0.5000
Epoch 7/1000
1/1 [==============================] - 0s 87ms/step - loss: 0.7091 - accuracy: 0.5000
Epoch 8/1000
1/1 [==============================] - 0s 27ms/step - loss: 0.7087 - accuracy: 0.5000
Epoch 9/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.7083 - accuracy: 0.5000
Epoch 10/1000
1/1 [==============================] - 0s 16ms/step - loss: 0.7079 - accuracy: 0.5000
Epoch 11/10

Evaluate the model on test set

In [10]:
model.evaluate(X_test_scaled, y_test)

1/1 [==============================] - 0s 334ms/step - loss: 0.5855 - accuracy: 0.6667


[0.5854937434196472, 0.6666666865348816]

In [11]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 199ms/step


array([[0.6447142 ],
       [0.56694895],
       [0.40433905],
       [0.5943529 ],
       [0.6498545 ],
       [0.6799389 ]], dtype=float32)

In [12]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

In [14]:
coef, intercept = model.get_weights()
coef, intercept

(array([[1.1257615],
        [0.7468925]], dtype=float32),
 array([-0.680116], dtype=float32))

This means w1 = 1.1257615, w2 = 0.7468925, bias = -0.680116

In [15]:
import math

def sigmoid(X):
    return 1 / (1 + math.exp(-X))

In [16]:
sigmoid(18)

0.9999999847700205

Instead of model.predict, we can write our own prediction function that uses w1, w2 and bias as parameters

In [17]:
def prediction_function(age, affordibility):
    w1 = coef[0]
    w2 = coef[1]
    b = intercept

    weighted_sum = w1 * age + w2 * affordibility + b
    return sigmoid(weighted_sum)

In [18]:
prediction_function(.18, 1)

/var/folders/6g/zbv6zp2s0yjfhrt83s_n6gj40000gn/T/ipykernel_95841/850877310.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1 / (1 + math.exp(-X))


0.5669489366110878

Now we start implementing gradient descent in plain python. Again the goal is to come up with same w1, w2 and bias that keras model calculated. We want to show how keras/tensorflow would have computed these values internally using gradient descent

First write couple of helper routines such as sigmoid and log_loss (binary_crossentropy)

In [20]:
def sigmoid_np(X):
    return 1 / (1 + np.exp(-X))

sigmoid_np(np.array([18, 0, 1]))

array([0.99999998, 0.5       , 0.73105858])

In [21]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

All right now comes the time to implement our final gradient descent function

In [23]:
def gradient_descent(age, affordability, y_true, epochs, loss_threshold):
    w1 = w2 = 1
    b = 0
    lr = 0.01
    n = len(age)

    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordability + b
        y_pred = sigmoid_np(weighted_sum)
        loss = log_loss(y_true, y_pred)

        w1_d = (1/n) * np.dot(np.transpose(age), (y_pred - y_true))
        w2_d = (1/n) * np.dot(np.transpose(affordability), (y_pred - y_true))
        b_d = np.mean(y_pred - y_true)

        w1 = w1 - lr * w1_d
        w2 = w2 - lr * w2_d
        b = b - lr * b_d

        print(f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{b}, loss:{loss}')

        if loss < loss_threshold:
            break

    return w1, w2, b

In [30]:
gradient_descent(X_train_scaled['age'], X_train_scaled['affordibility'], y_train, 1000, 0.6006)

Epoch:0, w1:0.9994981526694036, w2:0.9989669625078906, bias:-0.0022683735472737165, loss:0.7113403233723417
Epoch:1, w1:0.9989985401110477, w2:0.9979374183112477, bias:-0.004531110245838688, loss:0.7106947453938027
Epoch:2, w1:0.9985011596734701, w2:0.9969113648213787, bias:-0.006788217377105742, loss:0.7100526623879727
Epoch:3, w1:0.9980060086810597, w2:0.9958887993990542, bias:-0.009039702278981612, loss:0.7094140604774263
Epoch:4, w1:0.9975130844342297, w2:0.9948697193547812, bias:-0.011285572345427677, loss:0.7087789257803064
Epoch:5, w1:0.9970223842095907, w2:0.993854121949078, bias:-0.013525835026017508, loss:0.708147244410992
Epoch:6, w1:0.9965339052601244, w2:0.9928420043927506, bias:-0.015760497825493266, loss:0.7075190024807606
Epoch:7, w1:0.9960476448153585, w2:0.9918333638471725, bias:-0.01798956830332102, loss:0.7068941860984489
Epoch:8, w1:0.9955636000815411, w2:0.9908281974245644, bias:-0.020213054073245, loss:0.7062727813711064
Epoch:9, w1:0.995081768241817, w2:0.989826

Epoch:31, w1:0.9850353480845648, w2:0.9686608742467487, bias:-0.06983013416616679, loss:0.6928901601907568
Epoch:32, w1:0.9846035848800053, w2:0.9677380977412119, bias:-0.07192200428993396, loss:0.692346473724831
Epoch:33, w1:0.984173960498062, w2:0.9668187080451064, bias:-0.07400849637021605, loss:0.6918058528362555
Epoch:34, w1:0.983746471602547, w2:0.9659027010965188, bias:-0.07608961927322401, loss:0.6912682838010689
Epoch:35, w1:0.9833211148388618, w2:0.9649900727925357, bias:-0.07816538190718672, loss:0.6907337529095036
Epoch:36, w1:0.9828978868341801, w2:0.9640808189895617, bias:-0.08023579322189446, loss:0.690202246466511
Epoch:37, w1:0.9824767841976315, w2:0.9631749355036373, bias:-0.08230086220824277, loss:0.689673750792279
Epoch:38, w1:0.9820578035204839, w2:0.9622724181107587, bias:-0.08436059789777652, loss:0.6891482522227479
Epoch:39, w1:0.981640941376327, w2:0.9613732625471968, bias:-0.08641500936223429, loss:0.6886257371101183
Epoch:40, w1:0.9812261943212554, w2:0.96047

(0.9789596661653991, 0.8172535318988363, -0.7189729905887148)

In [31]:
coef, intercept

(array([[1.1257615],
        [0.7468925]], dtype=float32),
 array([-0.680116], dtype=float32))